# From Jupyter Notebook to Web Server: Automating Container Deployments with GitHub Actions

<strong>The purpose of this notebook is to provide an example of how one could use GitHub Actions to build a container image that could be hosted on the web via Helm charts. The hope is that what is learned can be applied to work that is interesting to you and open up avenues to create containers with interactive web content.</strong>

## Prerequisites

This walkthrough requires a [GitHub account](https://github.com/signup?ref_cta=Sign+up&ref_loc=header+logged+out&ref_page=%2F&source=header-home), a [Docker Hub account](https://hub.docker.com/signup), and for the example repository to be [forked](https://docs.github.com/en/pull-requests/collaborating-with-pull-requests/working-with-forks/fork-a-repo). 

## An Introduction to Containers

Containers package software and dependencies into a single image that can be run on any platform with a container engine installed. The image is portable and immutable, the software inside won't change. It will reliably run in the same state years from now making it powerful for easy reproducibility. 

The walkthrough from the June 20th, 2024 workshop covers what containers are in greater detail. For more information, see [From Jupyter Notebook to Web Server: Containerizing Interactive Visualizations](https://github.com/Gin-G/nbviz-to-container-gin-g/blob/main/June-20-2024/walkthrough.ipynb)

The container built during the June 20th workshop had to download data that caused timeouts for some users with slower internet connections. There is a Containerfile in this directory that contains a different web application that applies the same principles. It creates a smaller image that loads the website faster and should not timeout. Each command/layer is explained in comments inside the containerfile. 

## Getting Started

We are going to work out of the browser for majority of this workshop. GitHub provides an Integrated Development Environment (IDE) that can load any repository and allows for editing directly in a web browser. 

Open this repository, [https://github.com/NicholasCote/nbviz-to-container](nbviz-to-container), in your browser of choice. The body of the page contains a button in the upper right hand corner to Fork the repository. Click the Fork button to bring up the Create a new fork page. From here we can select an Owner, you, and name the repository. To avoid confusion in the future it's best to append the repository name with something like your GitHub user account name. All the other defaults are fine and we can select the Create fork button at the bottom of the page. This creates a copy of the repo contents under your own GitHub account. 

**Note:** If you attended the previous workshop we forked this repository and an existing fork may already exist. If you find yourself in the situation we can try to sync your existing fork or delete the old fork and create a new one. 

Once the fork is complete we will be on a GitHub page with the repository contents, but under our own accounts. The next step is going to be to open the repository in [github.dev](https://github.dev/). The easiest way to do this is to change the .com to .dev in the full URL displayed in our browser and hit Enter. We should now be in the GitHub IDE and can work inside the repository. 

## On-premise cloud deployment and automation

A working container image that runs a web server can be used in a number of different places, including as the content for a website at a live URL. Automation with GitHub Actions can be utilized to build a new image when specific repository changes are made and update configuration files, known as a [Helm chart](https://helm.sh/), to use the newest image. The Helm chart directory in the repository can be connected to a Continuous Delivery tool called [Argo CD](https://argoproj.github.io/cd/) which enables automatic synchronization. The NSF NCAR CISL On-premise Cloud Pilot is setup to host these kinds of workloads.

### GitHub Actions

A [GitHub Actions](https://docs.github.com/en/actions) workflow is defined as a YAML file and can be created to build a container when changes are made to a repository. This job can be run on GitHub-provided machines, called runners, or self-hosted runners that can be run on a users machine or a central computing system like the CISL On-premise Cloud Pilot. YAML files define each workflow and are stored at the repository's base in a directory named `.github/workflows/`. The workflow files can run scripts, basic commands, install additional software, and call jobs that are built and maintained by the GitHub Actions user community. The workflow we will define uses jobs that check out the repository and build & push the image without having to write out the full commands.  

#### Add `.github/workflows/` directory

On the left side bar in github.dev add a new folder named `.github` to the repository base by either right-clicking and selecting New Folder or clicking the new folder icon at the top of the sidebar. Inside the new `.github` folder add a new folder named `workflows`. This new location will be where we define how to run specific GitHub Actions. 

#### GitHub provided runner example

##### Build a container image

Sometimes it is advantageous to build an image, but not push it anywhere. For instance, you may want to test that the changes made don't affect the image build and it's still able to build completely without errors. We are going to start with this example before moving on to a workflow that pushes an image. Inside the `.github/workflows/` directory create a new file where we can define the GitHub Action. Use a descriptive name like `webapp-build-test.yaml` when creating the new file. The contents to place inside the file can be found below with explanations of each line, or group of lines, included as comments:

```yaml
# Name the Action
name: Automation to build a container image
# Define when the Action is run. 
# This example is run when there is a push to the August-08-2024/ directory on the main branch.
on:
  push:
    paths:
      - August-08-2024/**
    branches:
      - main
# Define the jobs to run. A job can have multiple steps and an Action can contain multiple jobs. 
jobs:
  build-image:
    # Use the latest ubuntu image to run the jobs
    # There are also windows and macos runners
    runs-on: ubuntu-latest
    steps:
      # Step 1 is to checkout the github repo used to build the Dockerfile
      - name: Check out the repo
        uses: actions/checkout@v4
      # Step 2 gets the date and sends it to GITHUB_OUTPUT to apply as an image tag
      - name: Get current date
        id: date
        run: echo "date=$(date +'%Y-%m-%d.%H')" >> $GITHUB_OUTPUT
      # Step 3 builds the docker image
      - name: Build Docker image
        uses: docker/build-push-action@v4
        with:
          # Provide the August-08-2024 directory as build context 
          context: August-08-2024/.
          # Specify where the Containerfile is located in relation to the repo base path
          file: August-08-2024/Containerfile
          # Disable the push to docker hub
          push: false
          # Provide the tags to apply to the image, this example uses the date from step 2
          tags: |
            ncote/workshop-webapp:${{ steps.date.outputs.date }}
```

```note
Remember to update the tag so that it uses your Docker Hub repository name.
```

Push the changes, the new workflow action file, by using the branch on the left side of your browser window. It should have a circle with a 1 inside it to show we have 1 change that can be committed and pushed. After clicking on the branch icon there will be a text box to enter a commit message. Enter a descriptive message like added build test workflow file and click the green Commit & Push button. Our Action is now live in our repository and we can use it to build images. 

##### Test the new workflow

The GitHub action defined only triggers when changes are made to the `August-08-2024` directory on the main branch. Let's open the Containerfile and add a new comment so it changes the Containerfile, but not actually how it's built and run. Once the comment has been added you will see that the GitHub extension in the left bar shows we have another change that can be pushed. Let's go ahead and add another commit message and push the new changes to kick off the GitHub Action. Once the push has been made we can watch the status of our GitHub action by going back to the repository URL and selecting Actions in the middle of the top navigation bar on the site.  

Once the job has completed successfully let's disable it from running. From the All workflows Actions page, on the left side select the individual job name, "Automation to build a container image". Once on this page, in the upper right there's a button with 3 dots. Click this to bring up a menu with 3 options, the bottom one being Disable workflow. Click the Disable workflow button to turn off the Action for now. 

##### Build and push a container image

Once we know the image is building correctly we can go ahead and push that image to a container registry so it can be shared with others or reused. The image created in the example is pushed to [Docker Hub](https://hub.docker.com/) and uses credentials stored as repository [Actions secrets](https://docs.github.com/en/actions/security-guides/using-secrets-in-github-actions) to log in. If we go back to the code repository in our browser we can open up the repository settings by using the far right option on the top navigation bar. Once in the repo security settings we can look at the left navigation menu that is opened and under Security there is a dropdown for Secrets and variables that contains a link to Actions. Once we have the Actions repository secrets open we can add our Docker Hub information based on the variables used in the workflow file, DOCKERHUB_USERNAME & DOCKERHUB_TOKEN. Inside the `.github/workflows/` directory create another file that will build and push the image using a descriptive name like `webapp-build-push.yaml`. The contents to place inside the file can be found below with explanations of each line, or group of lines, included as comments:

```yaml
# Name the Action
name: Automation to build a container image
# Define when the Action is run. This example is run when there is a push to the August-08-2024/ directory on the main branch.
on:
  push:
    paths:
      - August-08-2024/**
    branches:
      - main
# Define the jobs to run. A job can have multiple steps and an Action can contain multiple jobs. 
jobs:
  build-image:
    # Use the latest ubuntu image to run the jobs
    runs-on: ubuntu-latest
    steps:
      # Step 1 is to checkout the github repo used to build the Dockerfile
      - name: Check out the repo
        uses: actions/checkout@v4
      # Step 2 is to login to Docker Hub so the image can be pushed
      - name: Login to Docker Hub
        uses: docker/login-action@v2
        # GitHub repository secrets are used as variables to provide login information to Docker Hub
        # DOCKERHUB_USERNAME and DOCKERHUB_TOKEN need to be added to the Actions secrets
        with:
          username: ${{ secrets.DOCKERHUB_USERNAME }}
          password: ${{ secrets.DOCKERHUB_TOKEN }}
      # Step 3 gets the date and sends it to GITHUB_OUTPUT to apply as an image tag
      - name: Get current date
        id: date
        run: echo "date=$(date +'%Y-%m-%d.%H')" >> $GITHUB_OUTPUT
      # Step 4 builds and pushes the docker image
      - name: Build Docker image
        uses: docker/build-push-action@v4
        with:
          # Provide the August-08-2024 directory as build context 
          context: August-08-2024/.
          # Specify where the Containerfile is located in relation to the repo base path
          file: August-08-2024/Containerfile
          # Enable the push to docker hub
          push: true
          # Provide the tags to apply to the image, this example uses the latest image tag 
          tags: |
            ncote/workshop-webapp:${{ steps.date.outputs.date }}
```

```note
Remember to update the tag so that it uses your Docker Hub repository name.
```

Push the changes, the 2nd workflow action file, by using the branch on the left side of your browser window like we did before. Enter a descriptive message like added build and push workflow file and click the green Commit & Push button. Our Action is now live in our repository and we can use it to build and push images. 

##### Test the new workflow

Like before, the new workflow will only trigger when a change is made to the `August-08-2024` directory on the main branch. Let's add another comment, or delete the one we added before, to the Containerfile and push the changes to trigger the new Action.

We can watch the job progress from the Actions tab in our repository. Once it's successfully completed let's go to Docker Hub to see our image stored in our public repository with the date tag assigned by the workflow. 

If we want to test the image locally we could run a command like

`docker run -p 5006:5006 docker.io/ncote/workshop-webapp:2024-08-20`

We would want to make sure the repository name, image name, and image tag matched up with what was assigned during our workflow. 

#### Self-hosted runner example

GitHub self-hosted runners allow you to run GitHub Actions workflows on your own infrastructure, providing more control over the environment and resources used for continuous integration and deployment.

I have a custom container image that uses podman to do builds. I have included the Dockerfile in this repository as a reference in the `podman-runner/` directory. The up to date container image is also stored in my Docker Hub registry. In order to start the self-hosted runner, a GitHub API token is required when the container is launched along with the repository location. If you do not have an existing token, see [Personal access tokens](https://github.com/settings/tokens). The example below uses a local environment variable, `${GITHUB_RUNNER_TOKEN}`, instead of supplying the token as plain text in the command:

`docker run --privileged -e REPO={user/org}/{repo_name} -e TOKEN=${GITHUB_RUNNER_TOKEN} docker.io/ncote/podman-runner:2024-08-06`

Replace {user/org}/{repo_name} with the repository you want to connect the runner to, i.e. NicholasCote/nbviz-to-container. When run this will pull down the runner container image and start it automatically. There are a few prompts that will come up, the default value can be used for each one. Once those settings are entered there will be a confirmation that the runner is connected to GitHub and is Listening to Jobs. A GitHub action workflow that utilizes self-hosted runners can now be added and executed. Explanations of each line, or group of lines, are provided as comments in the example seen below:

```yaml
# This workflow builds container images and pushes them to Docker Hub
# Set the workflow name
name: Workshop self-hosted Build & Push

# Define the trigger that starts the action
# For this workflow the trigger is on a push that changes anything in the August-08-24 directory in the August-08-2024/ directory on the main branch
on:
  push:
    paths:
      - August-08-2024/**
    branches:
      - main

# Define the actions that are going to take place as part of this workflow    
jobs:
  # Name the job(s)
  build-push-sh-runner:
    # Define where the job should run
    # In this case it will be run on the self-hosted runner image
    runs-on: self-hosted
    # Set the steps to take in order
    steps:
      # Step 1 is to checkout the github repo used to build the container image
      - name: Check out the repo
        uses: actions/checkout@v4
      # Get the date to apply to image tag
      - name: Get current date
        id: date
        run: echo "date=$(date +'%Y-%m-%d.%H.%M')" >> $GITHUB_OUTPUT
      # Use podman to build the image and tag it with the current date
      - name: Build and push image
        run: podman build -f August-08-2024/Containerfile -t docker.io/ncote/workshop-webapp:${{ steps.date.outputs.date }} August-08-2024/.
      # Login to Docker Hub with the repository secrets defined earlier in the workshop
      - name: Login to Docker Hub
        run: podman login docker.io -u ${{ secrets.DOCKERHUB_USERNAME }} -p ${{ secrets.DOCKERHUB_TOKEN  }}
      # Push the image to Docker Hub
      - name: Push image to Docker Hub
        run: podman push docker.io/ncote/workshop-webapp:${{ steps.date.outputs.date }}
```

```note
Remember to update the tag so that it uses your Docker Hub repository name.
```

This workflow will run whenever there's changes to the `August-08-2024/` directory in the main branch and build a new container image. That image will be pushed to the Docker Hub container registry. If there is not a self-hosted runner connected the Action will stayed queued until one becomes available or it hits the timeout after 6 hours. 

##### Self hosted runner access

A self hosted runner would have access to other systems on the same network. For instance, the CISL Cloud Pilot hosts a container registry at [https://hub.k8s.ucar.edu]. It is currently only available to systems on the UCAR network. I can run a self hosted runner on my laptop while connected to the UCAR network and push my images to [https://hub.k8s.ucar.edu].

The CISL Cloud Pilot can connect and sync a Helm chart that defines a self hosted runner for a repository. 

### Helm charts

```note
Adding Helm charts to the CISL On-premise Cloud Pilot currently requires an administrator's assistance. Once it has been added a Continuous Delivery (CD) tool will automatically sync the application to match the Helm chart changes made to the git repository. 
```

In order to run the container on the CISL On-premise Cloud Pilot, a Kubernetes (K8s) cluster, a collection of 3 different K8s objects need to be created. The objects can be bundled together in to a Helm chart, a collection of templates that use custom values to define an applications. The 3 different K8s objects that are required are a [deployment](https://kubernetes.io/docs/concepts/workloads/controllers/deployment/), [service](https://kubernetes.io/docs/concepts/services-networking/service/), and [ingress](https://kubernetes.io/docs/concepts/services-networking/ingress/). A K8s deployment is the actual application running in 1 or more Pods. The K8s service exposes the Pod to the K8s internal network. Lastly the K8s ingress maps the service to a FQDN and exposes it outside the cluster. There are shared services running on the K8s cluster that will assign DNS records and valid certificates to provide a resolvable URL that is secured via HTTPS. In a Helm chart the K8s object definitions can be setup to take variables defined in a `values.yaml` file allowing for customization. 

#### Simple Web App Template

There is a simple web application Helm chart template available at this [link to web-app-helm repository](https://github.com/NicholasCote/web-app-helm). The `web-app-helm/` directory included in the repo can be copied in to the repository that contains the container image and application code. Inside the `web-app-helm/` directory is a file named `values.yaml` that needs to be updated with all the custom values required for your application. The `README.md` file included in the template repository contains instructions on what needs to be updated and what values to provide. A general overview of what is required to customize the `values.yaml` file is as follows:

1. A Name for your application, this is not the URL that will be deployed but the name of the k8s objects created
2. A URL path. This also is not the full URL, just the suffix you'd like to use after `.edu`. This is typically just `/` but may be things like `/api` that correspond to endpoints on your application.
3. An FQDN. This is the full URL for your application. Currently in the CISL cloud environment we only can create names under the `.k8s.ucar.edu` domain and your FQDN should end with `.k8s.ucar.edu`. Please make sure this is unique, try to browse to it before applying, and descriptive for your application. 
4. A unique name to store the TLS certificate assigned to the FQDN as a Kubernetes secret
5. Container image to use. This should be an image that is already built and has been pushed to a container registry that the application can pull from. By default it is set to look at docker.io so if you are using something different you need to specify that before your container registry and image name:tag
6. Container port to expose. Your containerized application will expose a port to the network in order to communicate. More often than not there is a default for the application you are using and you also have the ability to provide a specific port if you wanted. If you have run your container image locally it is usually in the URL you used to access it locally, ie. `http://127.0.0.1:8888` is running on port 8888 and would be the appropriate value to put in the Helm chart. 

### Using GitHub Actions to update `values.yaml`

We can put everything together and create a full CI/CD pipeline by updating the Helm values file at the end of our Actions workflow. Adding the code below to the end of our workflow files will update line 12 in the values file to use the image built in the workflow and push the changes to GitHub. 

```yaml
      - name: Update Helm values.yaml
        run: | 
          sed -i '12d' webapp-helm-chart/values.yaml
          echo "    image: docker.io/ncote/workshop-webapp:${{ steps.date.outputs.date }}" >> webapp-helm-chart/values.yaml
      - name: Commit changes to main branch
        run: |
          git config --global user.email "$GITHUB_ACTOR@users.noreply.github.com"
          git config --global user.name "$GITHUB_ACTOR"
          git commit -a -m "Update Helm chart via GH Actions"
          git push
```


If our Helm chart has been added to the CD tool, [Argo CD](https://argo-cd.readthedocs.io/en/stable/), when the Helm chart with the new image is pushed the application will be updated to use that new image within 3 minutes. 

## A note about CI/CD strategy

I wanted to end by discussing how we can combine a few of these approaches to create a full Continuous Integration Continuous Delivery (CI/CD) pipeline. There's multiple ways to approach this. I'm going to provide an example that's worked for me. 

I typically will work and test my changes out of a development branch before applying changes to the main branch. In order to protect the code in my main branch I use branch protection rules that don't allow direct pushes to the main branch and enforce Pull Requests (PRs). What I tend to do is create a container build job that runs when I change my container image file in the dev branch. This workflow only builds the container image and on a successful build it will create a PR to merge the changes made to the main branch. I will have a job in the main branch that builds the container image and pushes it to a container registry while also updating a Helm chart for the development application. Once Argo CD updates my test page I can browse to the site and confirm the changes I made to the application look good. At this point I can update the Helm chart for the production site to use the new image and create a PR to merge those changes so Argo CD will sync to the updates.  

Again, this is just one example that I've found works for my workflows. I can test my changes in different stages and not touch a production application until I have deployed all my changes to a test application and confirmed they work how I expected. 